In [ ]:
import os
os.chdir('/home/a001/Documents/ZhengHaoyu/python/1_important/OCMN')
import glob
from utils.generator import Generator, ERGenerator, BAGenerator, SFGenerator
from utils.utils import save_network, read_network, setup_logger, create_output_file
from utils.plot import transform_graphs, visualize_networks_with_bipartite, visualize_networks_with_matching_and_bipartite
from matching import Matching, MultiMatching
import config
import datetime
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import networkx as nx
import copy
import time

# Generate Synthetic Networks

In [ ]:
networks = ["base"] + [f"overlap={overlap}" for overlap in config.NETWORK_OVERLAP]

def generate_and_save_networks(k_range={"start": 2, "end": 10, "step": 0.1}):
    for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        for n in config.NETWORK_NODES_LIST:
            k = round(k, 2)
            
            er_generator = ERGenerator(n, k)
            er_graphs = er_generator.generate_networks(len(networks), config.NETWORK_OVERLAP)
            dir = f"ER_n={n}_k={k}"
            os.makedirs(os.path.join(config.SYNTHETIC_NET_PATH, 'ER', dir), exist_ok=True)
            for i in range(len(networks)):
                save_network(er_graphs[i], os.path.join(config.SYNTHETIC_NET_PATH, 'ER', dir, f"{networks[i]}.txt"))
            
            ba_generator = SFGenerator(n, k)
            ba_graphs = ba_generator.generate_networks(len(networks), config.NETWORK_OVERLAP)
            dir = f"BA_n={n}_k={k}"
            os.makedirs(os.path.join(config.SYNTHETIC_NET_PATH, 'BA', dir), exist_ok=True)
            for i in range(len(networks)):
                save_network(ba_graphs[i], os.path.join(config.SYNTHETIC_NET_PATH, 'BA', dir, f"{networks[i]}.txt"))

generate_and_save_networks()

# 不同网络类型、不同平均度k、不同节点数N、重复实验

In [ ]:
def optimization_amount(
    k_range={"start": 2, "end": 10, "step": 0.1},
    n_list=config.NETWORK_NODES_LIST,
    result_columns=[
        "network_type", "N", "<k>", "seq", 
        "MDS_1", "MDS_2", 
        "Diff_MDS_1", "Diff_MDS_2", "UDS_0", 
        "UDS_CLAPS", "UDS_RSU", "UDS_CLAPG", "UDS_ILP", 
        "clap_average_length", 
        "time_CLAPS", "time_RSU", "time_CLAPG", "time_ILP"]
):
    output_file_name = create_output_file(result_columns, "optimization_amount")

    for n in n_list:
        print(f"Processing n={n}...")

        # ER + ER; BA + BA; ER + BA
        for network_type in ["ER", "BA", "ER+BA"]:
            for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
                k = round(k, 2)
                for seq in range(10):
                    dir = f"{network_type}_n={n}_k={k}"
                    matchings = []

                    if "+" in network_type:
                        graph_1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, "ER", f"ER_n={n}_k={k}", f"base.txt"), n)
                        matching_1 = Matching(graph_1)
                        matching_1.HK_algorithm()
                        matchings.append(matching_1)
                        graph_2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, "BA", f"BA_n={n}_k={k}", f"base.txt"), n)
                        matching_2 = Matching(graph_2)
                        matching_2.HK_algorithm()
                        matchings.append(matching_2)
                    else:
                        for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir)):
                            # 包含base或-1的文件名
                            if "base" in file or "-1" in file:
                                graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir, file), n)
                                matching = Matching(graph)
                                matching.HK_algorithm()
                                matchings.append(matching)
                    
                    multi_matching = MultiMatching(matchings)
                    multi_matching_rsuu = copy.deepcopy(multi_matching)
                    multi_matching_glde = copy.deepcopy(multi_matching)
                    multi_matching_ilp = copy.deepcopy(multi_matching)

                    start_time = time.time()
                    pre_diff_mds_1_size, pre_diff_mds_2_size, pre_union_size, union_size, average_depth = multi_matching.CLAPS()
                    end_time = time.time()
                    time_clap_s = end_time - start_time
                    
                    start_time = time.time()
                    union_size_rsuu = multi_matching_rsuu.RSU()
                    end_time = time.time()
                    time_rsuu = end_time - start_time

                    start_time = time.time()
                    union_size_glde = multi_matching_glde.CLAPG()
                    end_time = time.time()
                    time_glde = end_time - start_time

                    start_time = time.time()
                    union_size_ilp = multi_matching_ilp.ILP_exact(budget_mode="auto")
                    end_time = time.time()
                    time_ilp = end_time - start_time
                    
                    with open(output_file_name, "a", encoding="utf-8") as output_file:
                        output_file.write(",".join([
                            f"{network_type}+{network_type}" if "+" not in network_type else network_type, str(n), str(k), str(seq), 
                            str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                            str(pre_diff_mds_1_size), str(pre_diff_mds_2_size), str(pre_union_size), 
                            str(union_size), str(union_size_rsuu), str(union_size_glde), str(union_size_ilp), 
                            str(average_depth),
                            str(time_clap_s),str(time_rsuu), str(time_glde), str(time_ilp)
                        ]) + "\n")

optimization_amount()

# 不同网络类型、不同平均度k、不同网络重叠程度

In [ ]:
def optimization_proportion(
    k_range={"start": 2, "end": 10, "step": 0.1},
    n_list=config.NETWORK_NODES_LIST,
    overlap_list=config.NETWORK_OVERLAP,
    result_columns=[
        "network_type", "N", "<k>", "overlap", 
        "MDS_1", "MDS_2", 
        "Diff_MDS_1", "Diff_MDS_2", "UDS_0", 
        "UDS_CLAPS", "UDS_RSU", "UDS_CLAPG", "UDS_ILP", 
        "clap_average_length", 
        "time_CLAPS", "time_RSU", "time_CLAPG", "time_ILP"]
):
    output_file_name = create_output_file(result_columns, "optimization_proportion")
    
    for n in n_list:
        print(f"Processing n={n}...")
        for network_type in ["ER", "BA"]:
            print(f"\tProcessing {network_type}...")
            for overlap in overlap_list:
                overlap = round(overlap, 3)
                if overlap < 0:
                    continue
                for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]), desc=f"\t\tOverlap={overlap}"):
                    k = round(k, 2)
                    dir = f"{network_type}_n={n}_k={k}"

                    matchings = []
                    for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir)):
                        if "base" in file or f"overlap={overlap}.txt" in file:
                            graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir, file), n)
                            matching = Matching(graph)
                            matching.HK_algorithm()
                            matchings.append(matching)
                    
                    multi_matching = MultiMatching(matchings)
                    multi_matching_rsuu = copy.deepcopy(multi_matching)
                    multi_matching_glde = copy.deepcopy(multi_matching)
                    multi_matching_ilp = copy.deepcopy(multi_matching)

                    start_time = time.time()
                    pre_diff_mds_1_size, pre_diff_mds_2_size, pre_union_size, union_size, average_depth = multi_matching.CLAPS()
                    end_time = time.time()
                    time_clap_s = end_time - start_time
                    
                    start_time = time.time()
                    union_size_rsuu = multi_matching_rsuu.RSU()
                    end_time = time.time()
                    time_rsuu = end_time - start_time

                    start_time = time.time()
                    union_size_glde = multi_matching_glde.CLAPG()
                    end_time = time.time()
                    time_glde = end_time - start_time

                    start_time = time.time()
                    union_size_ilp = multi_matching_ilp.ILP_exact(budget_mode="auto")
                    end_time = time.time()
                    time_ilp = end_time - start_time

                    with open(output_file_name, "a", encoding="utf-8") as output_file:
                        output_file.write(",".join([
                            f"{network_type}+{network_type}", str(n), str(k), str(overlap),
                            str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                            str(pre_diff_mds_1_size), str(pre_diff_mds_2_size), str(pre_union_size), 
                            str(union_size), str(union_size_rsuu), str(union_size_glde), str(union_size_ilp), 
                            str(average_depth),
                            str(time_clap_s),str(time_rsuu), str(time_glde), str(time_ilp)
                        ]) + "\n")

optimization_proportion()